In [ ]:
from dotenv import load_dotenv
import os
from src.services.data_fetch import DataFetch
from src.services.pipeline import MLPipeline
from src.core._mlflow import mlflow_init
load_dotenv()

from kaggle.api.kaggle_api_extended import KaggleApi  # noqa: E402


def run() -> None:
    """
    Run the entire pipeline.
    """
    kaggle_api_client = KaggleApi()
    _ = kaggle_api_client.authenticate()

    experiment = mlflow_init(
        experiment_name="Raisin",
        uri=os.getenv("MLFLOW_TRACKING_URI"),
    )
    
    data_fetch = DataFetch(kaggle_client=kaggle_api_client)
    dataset = data_fetch.fetch()

    pipeline = MLPipeline(dataset=dataset, experiment=experiment)
    pipeline.run_pipeline()


if __name__ == "__main__":
    run()

ModuleNotFoundError: No module named 'notebooks'